In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import warnings
from pathlib import Path

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')


##**Importar Datos**

In [2]:
df = pd.read_parquet('input/datos.parquet', engine='fastparquet')
df_dictionary = pd.read_excel('input/diccionario_variables.xlsx')

print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")

Dimensiones: 57366 filas × 675 columnas


In [3]:
df.head()

,ingreso_independientes,ingreso_agro,ingreso_pensiones,ingreso_remesas,edicion,lote,tipo,folio,viv,r101,...,r619_9,r619_10,r619_11,r619_11_otr,control620,r620_1,r620_2,r620_3,r620_4,r620_5
0,1000.0,6.25,NaN,NaN,2024.0,2.0,0.0,65.0,1.0,1.0,...,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,3.0,1.0
1,0.0,0.00,NaN,NaN,2024.0,2.0,0.0,65.0,1.0,2.0,...,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,3.0,1.0
2,0.0,0.00,645.0,NaN,2024.0,2.0,0.0,76.0,0.0,1.0,...,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,3.0,1.0
3,0.0,0.00,360.0,NaN,2024.0,2.0,0.0,80.0,0.0,1.0,...,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,3.0,3.0
4,0.0,0.00,NaN,10.0,2024.0,2.0,0.0,141.0,0.0,1.0,...,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
# Información general del dataset
print("-"*80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("-"*80)
print("\n")
df.info(verbose=True, show_counts=True)

--------------------------------------------------------------------------------
INFORMACIÓN GENERAL DEL DATASET
--------------------------------------------------------------------------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57366 entries, 0 to 57365
Data columns (total 675 columns):
 #    Column                  Non-Null Count  Dtype  
---   ------                  --------------  -----  
 0    ingreso_independientes  57366 non-null  float64
 1    ingreso_agro            57366 non-null  float64
 2    ingreso_pensiones       1150 non-null   float64
 3    ingreso_remesas         5198 non-null   float64
 4    edicion                 57366 non-null  float64
 5    lote                    57366 non-null  float64
 6    tipo                    57366 non-null  float64
 7    folio                   57366 non-null  float64
 8    viv                     57366 non-null  float64
 9    r101                    57366 non-null  float64
 10   idboleta                57366 non-null  

In [5]:
# Calcular valores faltantes por columna
missing_data = pd.DataFrame({
    'variable': df.columns,
    'n_faltantes': df.isnull().sum(),
    'pct_faltantes': (df.isnull().sum() / len(df)) * 100,
    'n_zeros': (df == 0).sum(),
    'pct_zeros': ((df == 0).sum() / len(df)) * 100,
    'dtype': df.dtypes
}).sort_values('pct_faltantes', ascending=False).reset_index(drop=True)

print("-"*80)
print("ANÁLISIS DE VALORES FALTANTES")
print("-"*80)
print(f"\nEstadísticas generales:")
print(f" - Variables sin faltantes: {(missing_data['pct_faltantes'] == 0).sum()}")
print(f" - Variables con <5% faltantes: {(missing_data['pct_faltantes'] < 5).sum()}")
print(f" - Variables con 5-20% faltantes: {((missing_data['pct_faltantes'] >= 5) & (missing_data['pct_faltantes'] < 20)).sum()}")
print(f" - Variables con 20-50% faltantes: {((missing_data['pct_faltantes'] >= 20) & (missing_data['pct_faltantes'] < 50)).sum()}")
print(f" - Variables con 50-95% faltantes: {((missing_data['pct_faltantes'] >= 50) & (missing_data['pct_faltantes'] < 95)).sum()}")
print(f" - Variables con >95% faltantes: {(missing_data['pct_faltantes'] >= 95).sum()}")
print(f" - Variables con >99% faltantes: {(missing_data['pct_faltantes'] >= 99).sum()}")
print("\n" + "-"*80)

--------------------------------------------------------------------------------
ANÁLISIS DE VALORES FALTANTES
--------------------------------------------------------------------------------

Estadísticas generales:
 - Variables sin faltantes: 140
 - Variables con <5% faltantes: 173
 - Variables con 5-20% faltantes: 68
 - Variables con 20-50% faltantes: 33
 - Variables con 50-95% faltantes: 221
 - Variables con >95% faltantes: 180
 - Variables con >99% faltantes: 110

--------------------------------------------------------------------------------


In [6]:
#Mostrar columnas que tengan mas del 95% de valores nulos
print('\n'+'-'*80)
print('Top de variables con mas del 95% de valores nulos')
print('-'*80)
top_variables_nulas = missing_data[missing_data['pct_faltantes'] > 95]
display(top_variables_nulas.head(30))

#Eliminar columnas con más del 95% de valores nulos
print('\n'+'-'*80)
print('Eliminando Columnas...')
df = df.drop(columns=top_variables_nulas['variable'].tolist())
print(f'Se han eliminado {len(top_variables_nulas['variable'].tolist())} columnas')
print('Dimensiones del dataset después de eliminar las columnas con más del 95% de valores nulos')
print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print('-'*80)



--------------------------------------------------------------------------------
Top de variables con mas del 95% de valores nulos
--------------------------------------------------------------------------------


,variable,n_faltantes,pct_faltantes,n_zeros,pct_zeros,dtype
0,r408otr,57366,100.000000,0,0.0,float64
1,r509cotr,57366,100.000000,0,0.0,float64
2,r218otr,57366,100.000000,0,0.0,float64
3,r217otro,57366,100.000000,0,0.0,float64
4,r214otr,57366,100.000000,0,0.0,float64
5,r01f_d,57366,100.000000,0,0.0,float64
6,r44010otr,57366,100.000000,0,0.0,float64
7,r01f_c,57366,100.000000,0,0.0,float64
8,r320otr_02,57366,100.000000,0,0.0,float64
9,r43512otr,57366,100.000000,0,0.0,float64



--------------------------------------------------------------------------------
Eliminando Columnas...
Se han eliminado 180 columnas
Dimensiones del dataset después de eliminar las columnas con más del 95% de valores nulos
Dimensiones: 57366 filas × 495 columnas
--------------------------------------------------------------------------------


In [7]:
#Verificacion de columnas constantes
print('-'*80)
print('Verificacion de columnas constantes')
print('-'*80)
const_cols = [col for col in df.columns if df[col].nunique() <= 1]
print(f'Total de columnas constantes: {len(const_cols)}')
print(f'Columnas constantes: {const_cols}')

print('\n'+ '-'*80)
print('Metricas de columnas constantes')
print('-'*80)
display(df[const_cols].describe())

#Eliminar columnas constantes
print('\n'+'-'*80)
print('Eliminando Columnas...')
df = df.drop(columns=const_cols)
print(f'Se han eliminado {len(const_cols)} columnas')
print('Dimensiones del dataset después de eliminar las columnas constantes')
print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print('-'*80)


--------------------------------------------------------------------------------
Verificacion de columnas constantes
--------------------------------------------------------------------------------
Total de columnas constantes: 7
Columnas constantes: ['edicion', 'r017', 'r019', 'r039', 'smin', 'r44010a', 'r447fm7']

--------------------------------------------------------------------------------
Metricas de columnas constantes
--------------------------------------------------------------------------------


,edicion,r017,r019,r039,smin,r44010a,r447fm7
count,57366.0,57366.0,57366.0,57366.0,5.736600e+04,49279.0,53737.0
mean,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0
std,0.0,0.0,0.0,0.0,1.136878e-13,0.0,0.0
min,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0
25%,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0
50%,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0
75%,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0
max,2024.0,1.0,1.0,1.0,3.630500e+02,0.0,0.0



--------------------------------------------------------------------------------
Eliminando Columnas...
Se han eliminado 7 columnas
Dimensiones del dataset después de eliminar las columnas constantes
Dimensiones: 57366 filas × 488 columnas
--------------------------------------------------------------------------------


In [8]:
# LIMPIEZA DE IDENTIFICADORES

# Estas columnas son únicas por fila o administrativas, no sirven para patrones
id_cols = ['idboleta', 'folio', 'lote', 'correlativo', 'r101', 'r015', 'r018', 'r020']
actual_ids = [c for c in id_cols if c in df.columns]

print(f'\nVariables administrativas identificadas: {len(actual_ids)}')
for col in actual_ids:
    n_unique = df[col].nunique()
    pct_unique = (n_unique / len(df)) * 100
    print(f'   - {col:15s}: {n_unique:6,} valores únicos ({pct_unique:.1f}%)')


print(f'\nEliminando {len(actual_ids)} variables administrativas...')
df = df.drop(columns=actual_ids)


Variables administrativas identificadas: 8
   - idboleta       : 18,058 valores únicos (31.5%)
   - folio          :    821 valores únicos (1.4%)
   - lote           :  1,658 valores únicos (2.9%)
   - correlativo    :  1,658 valores únicos (2.9%)
   - r101           :     14 valores únicos (0.0%)
   - r015           :     12 valores únicos (0.0%)
   - r018           :     10 valores únicos (0.0%)
   - r020           :      3 valores únicos (0.0%)

Eliminando 8 variables administrativas...


In [9]:
def detectar_columnas_gemelas(df):
    duplicadas = {} # Diccionario para guardar grupos {hash: [col1, col2, ...]}


    print("Generando huellas digitales de las columnas...")

    # Crear grupos sospechosos basados en el contenido
    hashes = {}
    for col in df.columns:
        # Creamos un hash tuple de la columna
        # Usamos los valores tal cual. Si hay nulos, se manejan bien.
        h = hash(tuple(df[col].values))
        if h not in hashes:
            hashes[h] = []
        hashes[h].append(col)

    # Verificar igualdad exacta y reportar
    grupos_identicos = []
    for h, cols in hashes.items():
        if len(cols) > 1:
            # Tenemos candidatos, verificamos el primero contra el resto
            base = cols[0]
            grupo = [base]
            for candidata in cols[1:]:
                if df[base].equals(df[candidata]):
                    grupo.append(candidata)
            if len(grupo) > 1:
                grupos_identicos.append(grupo)

    return grupos_identicos

In [10]:
# Ejecutar la función para detectar columnas gemelas
grupos_identicos = detectar_columnas_gemelas(df)

print(f"\nSe encontraron {len(grupos_identicos)} grupos de columnas idénticas.")
print("-" * 50)
mapa_etiquetas = dict(zip(df_dictionary['Variable'], df_dictionary['Etiqueta']))
print(f"\nGRUPOS DE DATOS IDÉNTICOS:")
print(f"{'Variable':<10} | {'Etiqueta'}")
print("-" * 80)

for grupo in grupos_identicos:
    for var in grupo:
        # Buscamos la etiqueta, si no existe ponemos un aviso
        etiqueta = mapa_etiquetas.get(var, "SIN ETIQUETA EN DICCIONARIO")

        print(f" {var:<10} : {etiqueta}")
    print("-" * 40)

print('\n'+'-'*80)
print('Eliminacion de columnas gemelas')
print('-'*80 + '\n')
#Eliminando una de las columnas gemelas
cols_a_eliminar = []

for g in grupos_identicos:
    # Lógica de selección: Nos quedamos con la que tiene el nombre más corto o 'mejor

    # Aquí ordenamos para que la primera sea la que se mantendra
    g_sorted = sorted(g, key=len)
    keep = g_sorted[0]
    drop = g_sorted[1:]

    cols_a_eliminar.extend(drop)

    print(f"Grupo idéntico: {g}")
    print(f" -> Mantener: '{keep}' | Eliminar: {drop}")
    print("-" * 20)

print(f"\nEliminando {len(cols_a_eliminar)} columnas redundantes...")
df = df.drop(columns=cols_a_eliminar)
print(f"Dimensiones tras deduplicación: {df.shape}")


Generando huellas digitales de las columnas...

Se encontraron 4 grupos de columnas idénticas.
--------------------------------------------------

GRUPOS DE DATOS IDÉNTICOS:
Variable   | Etiqueta
--------------------------------------------------------------------------------
 r021a      : Número de miembros del hogar
 miemh      : Número de miembros del hogar
----------------------------------------
 r021b      : Personas de 4 años y más del hogar
 r033       : Personas de 4 años y más del hogar
----------------------------------------
 r022       : Personas de 5 años y más del hogar
 r034       : Personas de 5 años y más del hogar
----------------------------------------
 r024       : Personas que se enfermaron del hogar
 r036       : Personas que se enfermaron del hogar
----------------------------------------

--------------------------------------------------------------------------------
Eliminacion de columnas gemelas
-------------------------------------------------------------

In [11]:
print('\n'+ '-'*80)
print('ELIMINAR COLUMNAS: >50% FALTANTES')
print('-'*80)
# Variables criticas que NO se eliminan aunque tengan >50% faltantes
vars_criticas = [
    # Económicas
    'ingreso_independientes', 'ingreso_agro', 'ingreso_remesas',
    'ingfa', 'ingpe', 'money', 'imei', 'imeds', 'imes', 'imnl', 'ingneto', 'ingre',
    'irefa', 'irefb', 'ires', 'totayuda',
    'gmed', 'gmvi', 'gmem', 'gmsa',
    'oia', 'oimed',

    # Empleo
    'actpr', 'actpr2012', 'actse',
    'ciuo414', 'ciiu416',
    'd411b', 'd412b', 'h411a', 'h412a',
    'segm',

    # Indicadores clave
    'pobreza', 'li',

    # Contexto
    'fac00', 'viv', 'znorte', 'tipo',

    # Demografía base
    'r021a', 'r021b', 'r022', 'r024', 'miemh', 'miemh_exdomestico',
]

# Identificar variables a eliminar
vars_a_eliminar = []
for col in df.columns:
    pct_null = df[col].isnull().mean()

    if pct_null > 0.50:  # Más del 50% faltantes
        # Verificar si NO está en la lista de críticas
        if col not in vars_criticas:
            vars_a_eliminar.append(col)

print(f"\nVariables con >50% faltantes: {sum(df[col].isnull().mean() > 0.50 for col in df.columns)}")
print(f"   De las cuales son críticas: {sum(col in vars_criticas and df[col].isnull().mean() > 0.50 for col in df.columns if col in df.columns)}")
print(f"   A eliminar: {len(vars_a_eliminar)}")

# Eliminar
df = df.drop(columns=vars_a_eliminar, errors='ignore')
print(f"\nEliminadas {len(vars_a_eliminar)} variables con >50% faltantes (no críticas)")

# Mostrar algunas de las eliminadas
if len(vars_a_eliminar) > 0:
    print(f"\nPrimeras 20 variables eliminadas:")
    for i, var in enumerate(vars_a_eliminar[:20], 1):
        pct = (df.columns.isin([var]).sum() == 0)  # Ya fue eliminada
        print(f"   {i:2d}. {var}")
    if len(vars_a_eliminar) > 20:
        print(f"   ... y {len(vars_a_eliminar) - 20} más")

print("\n" + "-"*80)
print(f"Dimensiones tras eliminacion de columnas: {df.shape}")


--------------------------------------------------------------------------------
ELIMINAR COLUMNAS: >50% FALTANTES
--------------------------------------------------------------------------------

Variables con >50% faltantes: 221
   De las cuales son críticas: 8
   A eliminar: 213

Eliminadas 213 variables con >50% faltantes (no críticas)

Primeras 20 variables eliminadas:
    1. r01b
    2. r01c
    3. r01d
    4. r01e
    5. r01f_a
    6. r06a
    7. r06b
    8. r06c
    9. r108a
   10. r108b
   11. r204
   12. r204g
   13. r205
   14. r209
   15. r210a
   16. r210b
   17. r210c
   18. r211a
   19. r211b
   20. r211c
   ... y 193 más

--------------------------------------------------------------------------------
Dimensiones tras eliminacion de columnas: (57366, 263)


In [12]:
print('\n'+ '-'*80)
print('TRANFORMAR columna ingreso_remesas')
print('-'*80)
if 'ingreso_remesas' in df.columns:
    print(f"\nTransformando ingreso_remesas...")

    # Crear binaria
    df['tiene_remesas'] = ((df['ingreso_remesas'].notna()) &
                           (df['ingreso_remesas'] > 0)).astype(int)

    # Crear monto (rellenar con 0)
    df['monto_remesas'] = df['ingreso_remesas'].fillna(0)

    # Eliminar original
    df = df.drop('ingreso_remesas', axis=1)

    print(f" Creadas: tiene_remesas (binaria) + monto_remesas (numérica)")
    print(f" Eliminada: ingreso_remesas (original)")

# Estadísticas
pct_recibe = (df['tiene_remesas'] == 1).sum() / len(df) * 100
monto_promedio = df[df['tiene_remesas'] == 1]['monto_remesas'].mean()
print(f"{pct_recibe:.1f}% de hogares reciben remesas")
print(f"Monto promedio (de los que reciben): ${monto_promedio:.2f}")


--------------------------------------------------------------------------------
TRANFORMAR columna ingreso_remesas
--------------------------------------------------------------------------------

Transformando ingreso_remesas...
 Creadas: tiene_remesas (binaria) + monto_remesas (numérica)
 Eliminada: ingreso_remesas (original)
9.1% de hogares reciben remesas
Monto promedio (de los que reciben): $194.88


In [13]:
# ----------------------------------------------------------------------------
# Verificar que NO eliminamos variables críticas por error
# ----------------------------------------------------------------------------
print("\nVERIFICACIÓN: Variables críticas preservadas")
print("="*80)

vars_criticas_check = [
    'ingfa', 'ingpe', 'pobreza', 'li', 'r021a',
    'actpr', 'fac00', 'viv'
]

todas_presentes = True
for var in vars_criticas_check:
    presente = var in df.columns
    status = "✅" if presente else "❌ ERROR"
    print(f"   {status} {var}")
    if not presente:
        todas_presentes = False

if todas_presentes:
    print(f"\nTodas las variables críticas están presentes")
else:
    print(f"\nALERTA: Algunas variables críticas faltan")

print("="*80)


VERIFICACIÓN: Variables críticas preservadas
   ✅ ingfa
   ✅ ingpe
   ✅ pobreza
   ✅ li
   ✅ r021a
   ✅ actpr
   ✅ fac00
   ✅ viv

Todas las variables críticas están presentes


## **FEATURE ENGINEERING**

In [14]:
print("="*80)
print('FEATURE ENGINEERING: VARIABLES ECONÓMICAS')
print("="*80)

# Dependencia de remesas
if 'ingfa' in df.columns and 'monto_remesas' in df.columns:
    df['dependencia_remesas'] = df['monto_remesas'] / (df['ingfa'] + 1)  # +1 para evitar división por 0
    df['dependencia_remesas'] = df['dependencia_remesas'].clip(0, 1)  # Limitar a [0,1]
    print("dependencia_remesas: % del ingreso total que son remesas")

# Diversificación de fuentes de ingreso
fuentes_ingreso = []
if 'ingreso_independientes' in df.columns:
    fuentes_ingreso.append((df['ingreso_independientes'] > 0).astype(int))
if 'ingreso_agro' in df.columns:
    fuentes_ingreso.append((df['ingreso_agro'] > 0).astype(int))
if 'tiene_remesas' in df.columns:
    fuentes_ingreso.append(df['tiene_remesas'])
if 'imeds' in df.columns:
    fuentes_ingreso.append((df['imeds'] > 0).astype(int))

if len(fuentes_ingreso) > 0:
    df['num_fuentes_ingreso'] = sum(fuentes_ingreso)
    print("num_fuentes_ingreso: Número de fuentes de ingreso activas (0-4)")

# Proporción de gastos
if all(col in df.columns for col in ['gmed', 'gmvi', 'gmem', 'gmsa', 'ingfa']):
    df['gasto_total_estimado'] = df['gmed'] + df['gmvi'] + df['gmem'] + df['gmsa']
    df['proporcion_gasto_ingreso'] = df['gasto_total_estimado'] / (df['ingfa'] + 1)
    df['proporcion_gasto_ingreso'] = df['proporcion_gasto_ingreso'].clip(0, 5)  # Limitar outliers
    print("gasto_total_estimado: Suma de gastos principales")
    print("proporcion_gasto_ingreso: Gastos / Ingresos")


print("\n" + "="*80)

FEATURE ENGINEERING: VARIABLES ECONÓMICAS
dependencia_remesas: % del ingreso total que son remesas
num_fuentes_ingreso: Número de fuentes de ingreso activas (0-4)
gasto_total_estimado: Suma de gastos principales
proporcion_gasto_ingreso: Gastos / Ingresos



In [15]:
print("="*80)
print('FEATURE ENGINEERING: VARIABLES DEMOGRÁFICAS')
print("="*80)

# Razón de dependencia demográfica
# Se necesitan variables de edad por grupos, pero con r021a, r022, r024 se pueden aproximar
if 'r021a' in df.columns and 'r022' in df.columns:
    # r021a = total miembros
    # r022 = personas 5+ años
    # Aproximación: menores de 5 años = r021a - r022
    df['menores_5'] = df['r021a'] - df['r022']
    df['menores_5'] = df['menores_5'].clip(lower=0)
    
    # Adultos = personas 5+ años (aproximación)
    df['adultos_aproximados'] = df['r022']
    
    # Razón de dependencia simplificada
    df['razon_dependencia_simple'] = df['menores_5'] / (df['adultos_aproximados'] + 1)
    print("menores_5: Estimación de menores de 5 años")
    print("razon_dependencia_simple: menores_5 / adultos")

# Proporción de personas enfermas
if 'r024' in df.columns and 'r021a' in df.columns:
    df['proporcion_enfermos'] = df['r024'] / (df['r021a'] + 1)
    df['proporcion_enfermos'] = df['proporcion_enfermos'].clip(0, 1)
    print("proporcion_enfermos: % de miembros que se enfermaron")

print("\n" + "="*80)

FEATURE ENGINEERING: VARIABLES DEMOGRÁFICAS
menores_5: Estimación de menores de 5 años
razon_dependencia_simple: menores_5 / adultos
proporcion_enfermos: % de miembros que se enfermaron



In [16]:
print("="*80)
print("FEATURE ENGINEERING: ÍNDICES COMPUESTOS")
print("="*80)

# Índice de servicios de vivienda (basado en r60*)
servicios_vivienda = ['r601', 'r602', 'r603', 'r604', 'r605', 'r606', 'r607', 'r608', 'r609']
servicios_disponibles = [col for col in servicios_vivienda if col in df.columns]

if len(servicios_disponibles) > 0:
    # Asumir que 1 = tiene servicio, 0 = no tiene
    df['indice_servicios'] = df[servicios_disponibles].sum(axis=1) / len(servicios_disponibles)
    print(f"indice_servicios: Proporción de servicios disponibles ({len(servicios_disponibles)} servicios)")

# Índice de victimización (basado en r619*)
victimizacion_vars = [f'r619_{i}' for i in range(1, 12) if f'r619_{i}' in df.columns]

if len(victimizacion_vars) > 0:
    df['indice_victimizacion'] = df[victimizacion_vars].sum(axis=1)
    df['fue_victima'] = (df['indice_victimizacion'] > 0).astype(int)
    print(f"indice_victimizacion: Número total de incidentes (0-11)")
    print(f"fue_victima: Binaria (1=sufrió al menos 1 incidente)")

# Índice de percepción de seguridad (basado en r620*)
seguridad_vars = ['r620_1', 'r620_2', 'r620_3', 'r620_4', 'r620_5']
seguridad_disponibles = [col for col in seguridad_vars if col in df.columns]

if len(seguridad_disponibles) > 0:
    # Asumir que valores más altos = menos seguridad
    df['indice_inseguridad'] = df[seguridad_disponibles].sum(axis=1)
    print(f"indice_inseguridad: Suma de restricciones por inseguridad")

# Índice de bienestar compuesto 
if all(col in df.columns for col in ['ingpe', 'pobreza', 'indice_servicios']):
    # Normalizar componentes
    ingpe_norm = (df['ingpe'] - df['ingpe'].min()) / (df['ingpe'].max() - df['ingpe'].min())
    pobreza_inv = 1 - df['pobreza']  # Invertir (no pobre = 1)
    
    df['indice_bienestar'] = (ingpe_norm + pobreza_inv + df['indice_servicios']) / 3
    print("indice_bienestar: Índice compuesto (ingreso + pobreza + servicios)")

print("\n" + "="*80)

FEATURE ENGINEERING: ÍNDICES COMPUESTOS
indice_servicios: Proporción de servicios disponibles (9 servicios)
indice_victimizacion: Número total de incidentes (0-11)
fue_victima: Binaria (1=sufrió al menos 1 incidente)
indice_inseguridad: Suma de restricciones por inseguridad
indice_bienestar: Índice compuesto (ingreso + pobreza + servicios)



In [17]:
print("="*80)
print("FEATURE ENGINEERING: VARIABLES DE EMPLEO")
print("="*80)

#Tasa de ocupación del hogar
if 'actpr' in df.columns and 'r021a' in df.columns:
    # actpr = 1 si está ocupado (verificar valores específicos)
    # Aproximación: contar cuántos están ocupados
    df['tiene_ocupado'] = ((df['actpr'].notna()) & (df['actpr'] > 0)).astype(int)
    print("tiene_ocupado: Al menos un miembro ocupado")

print("\n" + "="*80)

FEATURE ENGINEERING: VARIABLES DE EMPLEO
tiene_ocupado: Al menos un miembro ocupado



## **TRATAMIENTO DE VALORES FALTANTES**

In [18]:
print("-"*80)
print("ANÁLISIS DE VALORES FALTANTES")
print("-"*80)

# Calcular % faltantes por variable
missing_analysis = []
for col in df.columns:
    pct_null = df[col].isnull().mean() * 100
    if pct_null > 0:
        missing_analysis.append({
            'Variable': col,
            'Pct_Faltantes': pct_null,
            'N_Faltantes': df[col].isnull().sum(),
            'Tipo': df[col].dtype
        })

df_missing = pd.DataFrame(missing_analysis).sort_values('Pct_Faltantes', ascending=False)

print(f"\nVariables con faltantes: {len(df_missing)} de {len(df.columns)}")
print(f"Variables completas: {len(df.columns) - len(df_missing)}")

# Agrupar por rangos
print("\nDistribución de faltantes:")
if len(df_missing) > 0:
    print(f"   <5%:      {len(df_missing[df_missing['Pct_Faltantes'] < 5])}")
    print(f"   5-20%:    {len(df_missing[(df_missing['Pct_Faltantes'] >= 5) & (df_missing['Pct_Faltantes'] < 20)])}")
    print(f"   20-50%:   {len(df_missing[(df_missing['Pct_Faltantes'] >= 20) & (df_missing['Pct_Faltantes'] < 50)])}")
    print(f"   >50%:     {len(df_missing[df_missing['Pct_Faltantes'] >= 50])}")

    # Mostrar top 10 con más faltantes
    print("\nTop 10 variables con más faltantes:")
    print("-"*80)
    display(df_missing.head(10))
else:
    print("\nNo hay variables con faltantes")

print("-"*80)

--------------------------------------------------------------------------------
ANÁLISIS DE VALORES FALTANTES
--------------------------------------------------------------------------------

Variables con faltantes: 139 de 278
Variables completas: 139

Distribución de faltantes:
   <5%:      33
   5-20%:    66
   20-50%:   33
   >50%:     7

Top 10 variables con más faltantes:
--------------------------------------------------------------------------------


,Variable,Pct_Faltantes,N_Faltantes,Tipo
2,actse,71.817801,41199,float64
5,ciiu416,55.130217,31626,float64
4,ciuo414,55.130217,31626,float64
9,h412a,55.130217,31626,float64
8,h411a,55.130217,31626,float64
6,d411b,55.130217,31626,float64
7,d412b,55.130217,31626,float64
54,r4047,49.818708,28579,float64
53,r4046,49.818708,28579,float64
52,r4045,49.818708,28579,float64


--------------------------------------------------------------------------------


In [19]:

print("\n" + "="*80)
print("TRATAMIENTO DE VALORES FALTANTES")
print("="*80)

print("\nVariables de EMPLEO (faltantes = no ocupados)")
print("-"*80)

vars_empleo = [
    'ciuo414', 'ciiu416',           # Clasificaciones ocupacionales
    'd411b', 'd412b',               # Días trabajados
    'h411a', 'h412a',               # Horas trabajadas
    'segm',                         # Segmentación mercado
    'actse',                        # Estado ocupacional urbano
    'imeds',                        # Ingreso empleo dependiente
]

for var in vars_empleo:
    if var in df.columns:
        n_faltantes_antes = df[var].isnull().sum()
        if n_faltantes_antes > 0:
            # Para variables de empleo, los faltantes significan "No ocupado"
            if df[var].dtype in ['float64', 'int64']:
                # Numéricas: imputar con -1 (indicador de "No aplica")
                df[var] = df[var].fillna(-1)
                print(f"   ✅ {var:20s}: {n_faltantes_antes:6,} faltantes → -1 (No ocupado)")

print("\nVariables ECONÓMICAS (faltantes = $0)")
print("-"*80)

vars_economicas_imputar = [
    'ingreso_independientes', 'ingreso_agro',
    'imei', 'imes', 'imnl',
    'irefa', 'irefb', 'ires',
    'oia', 'oimed',
    'gmed', 'gmvi', 'gmem', 'gmsa',
]

for var in vars_economicas_imputar:
    if var in df.columns:
        n_faltantes_antes = df[var].isnull().sum()
        if n_faltantes_antes > 0:
            # Económicas: faltante = 0 (no recibe ese tipo de ingreso/gasto)
            df[var] = df[var].fillna(0)
            print(f"   {var:30s}: {n_faltantes_antes:6,} faltantes → 0")


print("\nVariables DERIVADAS (recalcular o imputar)")
print("-"*80)

vars_derivadas = [
    'dependencia_remesas', 'proporcion_gasto_ingreso', 
    'proporcion_enfermos', 'razon_dependencia_simple'
]

for var in vars_derivadas:
    if var in df.columns:
        n_faltantes_antes = df[var].isnull().sum()
        if n_faltantes_antes > 0:
            # Derivadas: imputar con 0 (ausencia del fenómeno)
            df[var] = df[var].fillna(0)
            print(f"   {var:30s}: {n_faltantes_antes:6,} faltantes → 0")


print("\nVariables NUMÉRICAS restantes (imputar con mediana por grupo)")
print("-"*80)

# Identificar variables numéricas que aún tienen faltantes
vars_numericas_pendientes = []
for col in df.columns:
    if df[col].dtype in ['float64', 'int64'] and df[col].isnull().sum() > 0:
        vars_numericas_pendientes.append(col)

if len(vars_numericas_pendientes) > 0:
    print(f"\n   Variables numéricas con faltantes: {len(vars_numericas_pendientes)}")
    
    # Estrategia: imputar con mediana por grupo de pobreza
    if 'pobreza' in df.columns:
        for var in vars_numericas_pendientes:
            n_faltantes_antes = df[var].isnull().sum()
            if n_faltantes_antes > 0:
                # Imputar con mediana por grupo de pobreza
                df[var] = df.groupby('pobreza')[var].transform(
                    lambda x: x.fillna(x.median())
                )
                
                # Si aún quedan faltantes (grupo sin datos), usar mediana global
                if df[var].isnull().sum() > 0:
                    df[var] = df[var].fillna(df[var].median())
                
                n_faltantes_despues = df[var].isnull().sum()
                print(f"{var:30s}: {n_faltantes_antes:6,} → {n_faltantes_despues:6,} faltantes")
    else:
        # Sin variable de pobreza, usar mediana global
        for var in vars_numericas_pendientes:
            n_faltantes_antes = df[var].isnull().sum()
            if n_faltantes_antes > 0:
                df[var] = df[var].fillna(df[var].median())
                print(f" {var:30s}: {n_faltantes_antes:6,} faltantes → mediana global")
else:
    print("   No hay variables numéricas pendientes")


print("\nVariables BINARIAS (imputar con 0 = ausencia)")
print("-"*80)

vars_binarias = [col for col in df.columns if df[col].nunique() <= 3 and df[col].isnull().sum() > 0]

for var in vars_binarias:
    n_faltantes_antes = df[var].isnull().sum()
    if n_faltantes_antes > 0:
        # Binarias: imputar con 0 (ausencia del fenómeno)
        df[var] = df[var].fillna(0)
        print(f"   {var:30s}: {n_faltantes_antes:6,} faltantes → 0")

# Verificación FINAL
total_faltantes_final = df.isnull().sum().sum()

print("\n" + "="*80)
print("VERIFICACIÓN FINAL DE VALORES FALTANTES")
print("="*80)

if total_faltantes_final == 0:
    print(f"\n¡PERFECTO! No hay valores faltantes en el dataset")
    print(f"Dataset completo: {df.shape[0]:,} filas × {df.shape[1]} columnas")
else:
    print(f"\nALERTA: Aún hay {total_faltantes_final:,} valores faltantes")
    print("\nVariables con faltantes:")
    for col in df.columns:
        n_faltantes = df[col].isnull().sum()
        if n_faltantes > 0:
            print(f"   - {col}: {n_faltantes:,} faltantes")

print("="*80)


TRATAMIENTO DE VALORES FALTANTES

Variables de EMPLEO (faltantes = no ocupados)
--------------------------------------------------------------------------------
   ✅ ciuo414             : 31,626 faltantes → -1 (No ocupado)
   ✅ ciiu416             : 31,626 faltantes → -1 (No ocupado)
   ✅ d411b               : 31,626 faltantes → -1 (No ocupado)
   ✅ d412b               : 31,626 faltantes → -1 (No ocupado)
   ✅ h411a               : 31,626 faltantes → -1 (No ocupado)
   ✅ h412a               : 31,626 faltantes → -1 (No ocupado)
   ✅ segm                : 28,211 faltantes → -1 (No ocupado)
   ✅ actse               : 41,199 faltantes → -1 (No ocupado)
   ✅ imeds               :  4,458 faltantes → -1 (No ocupado)

Variables ECONÓMICAS (faltantes = $0)
--------------------------------------------------------------------------------

Variables DERIVADAS (recalcular o imputar)
--------------------------------------------------------------------------------

Variables NUMÉRICAS restantes (imp


   Variables numéricas con faltantes: 130
actpr                         :  3,629 →      0 faltantes
actpr2012                     :  3,629 →      0 faltantes
aproba1                       :  2,800 →      0 faltantes
r105m                         :  1,317 →      0 faltantes
r107                          :  9,984 →      0 faltantes
r202a                         :  2,800 →      0 faltantes
r202b                         : 10,650 →      0 faltantes
r203                          :  2,800 →      0 faltantes
r212                          : 16,392 →      0 faltantes
r213                          : 22,102 →      0 faltantes
r214                          : 22,102 →      0 faltantes
r215                          : 22,451 →      0 faltantes
r216a                         : 26,728 →      0 faltantes
r217                          : 22,447 →      0 faltantes
r219                          : 16,392 →      0 faltantes
r301c                         :    236 →      0 faltantes
r302b                        

## **SELECCION DE VARIABLES**

In [20]:
print("\n" + "="*80)
print("SEPARACIÓN DE VARIABLES ESPECIALES")
print("="*80)

# Guardar variables que NO van al clustering
vars_no_clustering = []

if 'fac00' in df.columns:
    factor_expansion = df['fac00'].copy()
    vars_no_clustering.append('fac00')
    print(f"  fac00 separado (factor de expansión)")

# Dataset para clustering (sin fac00 ni IDs)
df_clustering = df.drop(columns=vars_no_clustering, errors='ignore')

print(f"\nVariables para clustering: {df_clustering.shape[1]}")
print(f"Variables separadas: {len(vars_no_clustering)}")


SEPARACIÓN DE VARIABLES ESPECIALES
  fac00 separado (factor de expansión)

Variables para clustering: 277
Variables separadas: 1


In [21]:
print("\n" + "="*80)
print("ANÁLISIS DE VARIANZA")
print("="*80)

# Calcular varianza de cada variable
variance_analysis = []
for col in df_clustering.columns:
    if df_clustering[col].dtype in ['float64', 'int64']:
        var = df_clustering[col].var()
        std = df_clustering[col].std()
        cv = (std / df_clustering[col].mean()) if df_clustering[col].mean() != 0 else 0
        
        variance_analysis.append({
            'Variable': col,
            'Varianza': var,
            'Std': std,
            'CV': cv,  # Coeficiente de variación
            'Min': df_clustering[col].min(),
            'Max': df_clustering[col].max()
        })

df_variance = pd.DataFrame(variance_analysis).sort_values('Varianza', ascending=True)

# Identificar variables con varianza muy baja o nula
vars_sin_varianza = df_variance[df_variance['Varianza'] < 0.0001]['Variable'].tolist()
vars_baja_varianza = df_variance[
    (df_variance['Varianza'] >= 0.0001) & 
    (df_variance['Varianza'] < 0.01)
]['Variable'].tolist()

print(f"\nAnálisis de varianza:")
print(f"   Variables sin varianza (<0.0001):  {len(vars_sin_varianza)}")
print(f"   Variables baja varianza (<0.01):   {len(vars_baja_varianza)}")

if len(vars_sin_varianza) > 0:
    print(f"\nVariables SIN varianza (eliminar):")
    for var in vars_sin_varianza[:10]:
        val_unico = df_clustering[var].unique()[0] if df_clustering[var].nunique() == 1 else "varios"
        print(f"   - {var:30s}: valor= {val_unico}")
    
    # Eliminar
    df_clustering = df_clustering.drop(columns=vars_sin_varianza)
    print(f"\nEliminadas {len(vars_sin_varianza)} variables sin varianza")

if len(vars_baja_varianza) > 0:
    print(f"\nVariables con BAJA varianza (revisar):")
    for var in vars_baja_varianza[:30]:
        print(f"   - {var}")

print(f"\nVariables restantes: {df_clustering.shape[1]}")

vars_eliminar = ['r4045', 'r4046', 'r44007b', 'r44010b', 'r319a4', 'r02a']

print(f"\nVariables a eliminar (>99.9% constante):")
for var in vars_eliminar:
    if var in df_clustering.columns:
        pct = (df_clustering[var] == df_clustering[var].mode()[0]).sum() / len(df_clustering) * 100
        print(f"  {var:15s}: {pct:.3f}% constante")

# Eliminar
df_clustering = df_clustering.drop(columns=vars_eliminar, errors='ignore')

print(f"\nEliminadas: {len(vars_eliminar)} variables")
print(f"Variables restantes: {df_clustering.shape[1]}")


ANÁLISIS DE VARIANZA

Análisis de varianza:
   Variables sin varianza (<0.0001):  5
   Variables baja varianza (<0.01):   29

Variables SIN varianza (eliminar):
   - r4047                         : valor= varios
   - r4041_1                       : valor= varios
   - r44818                        : valor= varios
   - r4044                         : valor= varios
   - r4043                         : valor= varios

Eliminadas 5 variables sin varianza

Variables con BAJA varianza (revisar):
   - r4045
   - r4046
   - r619_11
   - r32316a
   - r4049
   - r4041
   - r4042
   - r44817
   - r319a4
   - r02a
   - r44010b
   - r619_9
   - r4048
   - r319a3
   - r619_10
   - r619_3
   - fue_victima
   - r619_8
   - r44007b
   - r3213a1
   - r216a
   - r32310a
   - r619_5
   - r32307b
   - r44816
   - r619_6
   - r32313b
   - r616_05
   - r619_7

Variables restantes: 272

Variables a eliminar (>99.9% constante):
  r4045          : 99.988% constante
  r4046          : 99.981% constante
  r44007b 

In [22]:
print("\n" + "="*80)
print("ANÁLISIS DE CORRELACIONES")
print("="*80)

print(f"\nCalculando matriz de correlaciones ({df_clustering.shape[1]} variables)...")

# Calcular matriz de correlación
corr_matrix = df_clustering.corr()

print(f" Matriz calculada: {corr_matrix.shape[0]} × {corr_matrix.shape[1]}")

# Identificar pares de variables altamente correlacionadas
print(f"\nIdentificando variables altamente correlacionadas...")

# Umbral de correlación
CORR_THRESHOLD = 0.95

# Encontrar pares correlacionados
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) >= CORR_THRESHOLD:
            high_corr_pairs.append({
                'Var1': corr_matrix.columns[i],
                'Var2': corr_matrix.columns[j],
                'Correlacion': corr_matrix.iloc[i, j]
            })

df_high_corr = pd.DataFrame(high_corr_pairs).sort_values('Correlacion', ascending=False)

print(f"\nPares con correlación >= {CORR_THRESHOLD}: {len(df_high_corr)}")

if len(df_high_corr) > 0:
    print(f"\nTop 20 pares más correlacionados:")
    print("-"*80)
    display(df_high_corr.head(20))
else:
    print(f"\n No hay pares con correlación >= {CORR_THRESHOLD}")


ANÁLISIS DE CORRELACIONES

Calculando matriz de correlaciones (266 variables)...


 Matriz calculada: 266 × 266

Identificando variables altamente correlacionadas...

Pares con correlación >= 0.95: 16

Top 20 pares más correlacionados:
--------------------------------------------------------------------------------


,Var1,Var2,Correlacion
0,ingreso_independientes,imei,1.000000
5,r022,adultos_aproximados,1.000000
1,ingreso_agro,ingneto,1.000000
3,r021a,miemh_exdomestico,0.999485
8,actpr,actpr2012,0.998891
12,irefa,totayuda,0.998561
7,r033,adultos_aproximados,0.986985
4,r022,r033,0.986985
9,actpr,r403,0.966267
2,r021a,r033,0.962894


In [23]:
print("\n" + "="*80)
print("ELIMINACIÓN DE VARIABLES REDUNDANTES")
print("="*80)

if len(df_high_corr) > 0:
    print(f"\n Estrategia: De cada par, eliminar la variable con menor varianza")
    
    vars_a_eliminar = set()
    
    for idx, row in df_high_corr.iterrows():
        var1 = row['Var1']
        var2 = row['Var2']
        
        # Si ninguna ya fue eliminada
        if var1 not in vars_a_eliminar and var2 not in vars_a_eliminar:
            # Calcular varianza de cada una
            var1_variance = df_clustering[var1].var()
            var2_variance = df_clustering[var2].var()
            
            # Eliminar la de menor varianza
            if var1_variance < var2_variance:
                vars_a_eliminar.add(var1)
                print(f"   {var1:30s} correlacionada con {var2} (menor varianza)")
            else:
                vars_a_eliminar.add(var2)
                print(f"   {var2:30s} correlacionada con {var1} (menor varianza)")
    
    print(f"\nVariables a eliminar por alta correlación: {len(vars_a_eliminar)}")
    
    # Eliminar
    df_clustering = df_clustering.drop(columns=list(vars_a_eliminar))
    print(f"Variables eliminadas: {len(vars_a_eliminar)}")
    print(f"Variables restantes: {df_clustering.shape[1]}")
else:
    print(f"\nNo hay variables redundantes que eliminar")


ELIMINACIÓN DE VARIABLES REDUNDANTES

 Estrategia: De cada par, eliminar la variable con menor varianza
   imei                           correlacionada con ingreso_independientes (menor varianza)
   adultos_aproximados            correlacionada con r022 (menor varianza)
   ingreso_agro                   correlacionada con ingneto (menor varianza)
   r021a                          correlacionada con miemh_exdomestico (menor varianza)
   actpr2012                      correlacionada con actpr (menor varianza)
   irefa                          correlacionada con totayuda (menor varianza)
   r022                           correlacionada con r033 (menor varianza)
   r403                           correlacionada con actpr (menor varianza)
   r033                           correlacionada con miemh_exdomestico (menor varianza)
   money                          correlacionada con ingre (menor varianza)
   oia                            correlacionada con r44105 (menor varianza)
   fue_victima

In [24]:
print('\n'+ '-'*80)
print("CLASIFICACIÓN DE VARIABLES")
print('-'*80)

print("\nClasificando variables...")

categorias = {
    'economicas': [],
    'demograficas': [],
    'empleo': [],
    'educacion': [],
    'salud': [],
    'gastos': [],
    'vivienda': [],
    'servicios': [],
    'seguridad': [],
    'indices': [],
    'otras': []
}

for var in df_clustering.columns:
    # Económicas
    if any(k in var.lower() for k in ['ingreso', 'ingpe', 'ingfa', 'gm', 'remesas', 'monto', 'oimed', 'totayuda', 'imes', 'imnl', 'ingneto', 'irefb', 'ires']):
        categorias['economicas'].append(var)
    # Demográficas
    elif var in ['r024', 'miemh_exdomestico', 'menores_5', 'razon_dependencia_simple', 'proporcion_enfermos', 'r01a']:
        categorias['demograficas'].append(var)
    # Empleo
    elif any(k in var.lower() for k in ['actpr', 'ciuo', 'ciiu', 'd411', 'd412', 'h411', 'h412', 'segm', 'tiene_ocupado']):
        categorias['empleo'].append(var)
    # Educación
    elif var.startswith(('r10', 'r20')) or var == 'aproba1':
        categorias['educacion'].append(var)
    # Salud (r30*, r31* pero NO r32*)
    elif var.startswith('r3') and not var.startswith('r32'):
        categorias['salud'].append(var)
    # Gastos (r32*)
    elif var.startswith('r32'):
        categorias['gastos'].append(var)
    # Vivienda (r44*, r40*, r447*)
    elif var.startswith(('r44', 'r40', 'r45')):
        categorias['vivienda'].append(var)
    # Servicios (r60*)
    elif var.startswith('r60'):
        categorias['servicios'].append(var)
    # Seguridad (r61*, control*, indice_victimizacion, indice_inseguridad)
    elif var.startswith(('r61', 'control')) or 'victimizacion' in var or 'inseguridad' in var:
        categorias['seguridad'].append(var)
    # Índices y contexto
    elif var in ['li', 'pobreza', 'tipo', 'viv', 'znorte'] or var.startswith('indice'):
        categorias['indices'].append(var)
    else:
        categorias['otras'].append(var)

# Reporte
print("\nClasificación actual:")
for cat, vars_list in categorias.items():
    print(f"   {cat:15s}: {len(vars_list):3d} variables")

total_clasificadas = sum(len(v) for v in categorias.values())
print(f"\n   TOTAL: {total_clasificadas} de {len(df_clustering.columns)} variables")



--------------------------------------------------------------------------------
CLASIFICACIÓN DE VARIABLES
--------------------------------------------------------------------------------

Clasificando variables...

Clasificación actual:
   economicas     :  20 variables
   demograficas   :   6 variables
   empleo         :   8 variables
   educacion      :   9 variables
   salud          :  27 variables
   gastos         :  63 variables
   vivienda       :  54 variables
   servicios      :   9 variables
   seguridad      :  27 variables
   indices        :   7 variables
   otras          :  23 variables

   TOTAL: 253 de 253 variables


## **REDUCCION DE VARIABLES**

In [25]:
print("\n" + "-"*80)
print("REDUCCIÓN: GASTOS")
print("-"*80)

gastos_vars = categorias['gastos']

print(f"\n   Variables de gastos actuales: {len(gastos_vars)}")

# Calcular varianza de cada variable de gastos
gastos_variance = []
for var in gastos_vars:
    var_val = df_clustering[var].var()
    mean_val = df_clustering[var].mean()
    cv = df_clustering[var].std() / mean_val if mean_val > 0 else 0
    
    gastos_variance.append({
        'Variable': var,
        'Varianza': var_val,
        'CV': cv,
        'Media': mean_val
    })

df_gastos = pd.DataFrame(gastos_variance).sort_values('CV', ascending=False)

# Mantener top 25 por coeficiente de variación
gastos_mantener = df_gastos.head(25)['Variable'].tolist()

# Asegurar que r328total está incluido (gasto total)
if 'r328total' in gastos_vars and 'r328total' not in gastos_mantener:
    gastos_mantener.append('r328total')

gastos_eliminar = [v for v in gastos_vars if v not in gastos_mantener]

print(f"Mantener: {len(gastos_mantener)} variables (mayor variabilidad)")
print(f"Eliminar: {len(gastos_eliminar)} variables")

print("\n" + "-"*80)
print("REDUCCIÓN: VIVIENDA")
print("-"*80)

vivienda_vars = categorias['vivienda']

print(f"\n   Variables de vivienda actuales: {len(vivienda_vars)}")

# Calcular varianza de cada variable de vivienda
vivienda_variance = []
for var in vivienda_vars:
    var_val = df_clustering[var].var()
    mean_val = df_clustering[var].mean()
    cv = df_clustering[var].std() / mean_val if mean_val > 0 else 0
    
    vivienda_variance.append({
        'Variable': var,
        'Varianza': var_val,
        'CV': cv,
        'Media': mean_val
    })

df_vivienda = pd.DataFrame(vivienda_variance).sort_values('CV', ascending=False)

# Mantener top 25 por coeficiente de variación
vivienda_mantener = df_vivienda.head(25)['Variable'].tolist()

vivienda_eliminar = [v for v in vivienda_vars if v not in vivienda_mantener]

print(f"Mantener: {len(vivienda_mantener)} variables (mayor variabilidad)")
print(f"Eliminar: {len(vivienda_eliminar)} variables")

print("\n" + "-"*80)
print("REDUCCIÓN: SALUD")
print("-"*80)

salud_vars = categorias['salud']

print(f"\nVariables de salud actuales: {len(salud_vars)}")

# Calcular varianza
salud_variance = []
for var in salud_vars:
    var_val = df_clustering[var].var()
    cv = df_clustering[var].std() / df_clustering[var].mean() if df_clustering[var].mean() > 0 else 0
    
    salud_variance.append({
        'Variable': var,
        'Varianza': var_val,
        'CV': cv
    })

df_salud = pd.DataFrame(salud_variance).sort_values('CV', ascending=False)

# Mantener top 18
salud_mantener = df_salud.head(18)['Variable'].tolist()
salud_eliminar = [v for v in salud_vars if v not in salud_mantener]

print(f"Mantener: {len(salud_mantener)} variables")
print(f"Eliminar: {len(salud_eliminar)} variables")

print("\n" + "="*80)
print("REDUCCIÓN: SEGURIDAD")
print("="*80)

seguridad_vars = categorias['seguridad']

print(f"\n Variables de seguridad actuales: {len(seguridad_vars)}")

# Mantener variables críticas de victimización
vars_victimizacion = [v for v in seguridad_vars if v.startswith('r619')]

# Calcular varianza del resto
otras_seguridad = [v for v in seguridad_vars if not v.startswith('r619')]

seguridad_variance = []
for var in otras_seguridad:
    var_val = df_clustering[var].var()
    seguridad_variance.append({
        'Variable': var,
        'Varianza': var_val
    })

df_seg = pd.DataFrame(seguridad_variance).sort_values('Varianza', ascending=False)

# Mantener r619* (11 vars) + top 7 del resto
seguridad_mantener = vars_victimizacion + df_seg.head(7)['Variable'].tolist()
seguridad_eliminar = [v for v in seguridad_vars if v not in seguridad_mantener]

print(f" Mantener: {len(seguridad_mantener)} variables")
print(f" Eliminar: {len(seguridad_eliminar)} variables")

print("\n" + "="*80)
print("APLICANDO REDUCCIONES")
print("="*80)

vars_a_eliminar_total = gastos_eliminar + vivienda_eliminar + salud_eliminar + seguridad_eliminar

print(f"\nVariables a eliminar:")
print(f"   Gastos:      {len(gastos_eliminar)}")
print(f"   Vivienda:    {len(vivienda_eliminar)}")
print(f"   Salud:       {len(salud_eliminar)}")
print(f"   Seguridad:   {len(seguridad_eliminar)}")
print('-'*40)
print(f"   TOTAL:       {len(vars_a_eliminar_total)}")

# Eliminar
df_clustering = df_clustering.drop(columns=vars_a_eliminar_total)

print(f"\nVariables eliminadas: {len(vars_a_eliminar_total)}")
print(f"Variables restantes: {df_clustering.shape[1]}")



--------------------------------------------------------------------------------
REDUCCIÓN: GASTOS
--------------------------------------------------------------------------------

   Variables de gastos actuales: 63
Mantener: 25 variables (mayor variabilidad)
Eliminar: 38 variables

--------------------------------------------------------------------------------
REDUCCIÓN: VIVIENDA
--------------------------------------------------------------------------------

   Variables de vivienda actuales: 54
Mantener: 25 variables (mayor variabilidad)
Eliminar: 29 variables

--------------------------------------------------------------------------------
REDUCCIÓN: SALUD
--------------------------------------------------------------------------------

Variables de salud actuales: 27
Mantener: 18 variables
Eliminar: 9 variables

REDUCCIÓN: SEGURIDAD

 Variables de seguridad actuales: 27
 Mantener: 18 variables
 Eliminar: 9 variables

APLICANDO REDUCCIONES

Variables a eliminar:
   Gastos:     

In [26]:
print("\n" + "-"*80)
print("BALANCE FINAL")
print("-"*80)

# Reclasificar
balance_final = {
    'Económicas': len([v for v in df_clustering.columns if v in categorias['economicas']]),
    'Demográficas': len([v for v in df_clustering.columns if v in categorias['demograficas']]),
    'Empleo': len([v for v in df_clustering.columns if v in categorias['empleo']]),
    'Educación': len([v for v in df_clustering.columns if v in categorias['educacion']]),
    'Salud': len([v for v in df_clustering.columns if v in categorias['salud']]),
    'Gastos': len([v for v in df_clustering.columns if v in categorias['gastos']]),
    'Vivienda': len([v for v in df_clustering.columns if v in categorias['vivienda']]),
    'Servicios': len([v for v in df_clustering.columns if v in categorias['servicios']]),
    'Seguridad': len([v for v in df_clustering.columns if v in categorias['seguridad']]),
    'Índices/Contexto': len([v for v in df_clustering.columns if v in categorias['indices']]),
    'Otras': len([v for v in df_clustering.columns if v in categorias['otras']])
}

print("\nDISTRIBUCIÓN BALANCEADA:\n")
total_final = 0
for cat, count in balance_final.items():
    pct = (count / df_clustering.shape[1]) * 100
    total_final += count
    print(f"   {cat:20s}: {count:3d} vars ({pct:5.1f}%)")

print(f"\n   {'TOTAL':20s}: {total_final:3d} vars")

# Verificar balance
gastos_vivienda_pct = ((balance_final['Gastos'] + balance_final['Vivienda']) / df_clustering.shape[1]) * 100
economicas_pct = (balance_final['Económicas'] / df_clustering.shape[1]) * 100

print(f"\nMÉTRICAS DE BALANCE:")
print(f"   Gastos + Vivienda: {gastos_vivienda_pct:.1f}% (antes: 45%)")
print(f"   Económicas:        {economicas_pct:.1f}% (antes: 6%)")


print("-"*80)


--------------------------------------------------------------------------------
BALANCE FINAL
--------------------------------------------------------------------------------

DISTRIBUCIÓN BALANCEADA:

   Económicas          :  20 vars ( 11.9%)
   Demográficas        :   6 vars (  3.6%)
   Empleo              :   8 vars (  4.8%)
   Educación           :   9 vars (  5.4%)
   Salud               :  18 vars ( 10.7%)
   Gastos              :  25 vars ( 14.9%)
   Vivienda            :  25 vars ( 14.9%)
   Servicios           :   9 vars (  5.4%)
   Seguridad           :  18 vars ( 10.7%)
   Índices/Contexto    :   7 vars (  4.2%)
   Otras               :  23 vars ( 13.7%)

   TOTAL               : 168 vars

MÉTRICAS DE BALANCE:
   Gastos + Vivienda: 29.8% (antes: 45%)
   Económicas:        11.9% (antes: 6%)
--------------------------------------------------------------------------------


In [27]:
# Guardar dataset final
output_path = 'output/dataset_seleccionado_para_clustering.parquet'

df_clustering.to_parquet(output_path, compression='snappy', engine='fastparquet', index=False)
print(f"\nDataset guardado:")
print(f"  {output_path}")
print(f"  {df_clustering.shape[0]:,} hogares × {df_clustering.shape[1]} variables")



Dataset guardado:
  output/dataset_seleccionado_para_clustering.parquet
  57,366 hogares × 168 variables
